In [1]:
from PIL import Image
import os
import math
import glob
import pandas as pd
import cv2
import numpy as np

In [2]:
imageFolder = "images"
debugImagesFolder = "debugImages"

png_dosyalari = [dosya for dosya in os.listdir(imageFolder) if dosya.endswith('.png')]

konumlar = {}

for png_dosyasi in png_dosyalari:
    dosya_adi = os.path.splitext(png_dosyasi)[0]
    
    konum_tur = dosya_adi.rsplit('_', 1)
    
    if konum_tur[1] not in konumlar:
        konumlar[konum_tur[1]] = []

    # Dosya adını ve konum-tür bilgisini sözlüğe ekle
    konumlar[konum_tur[1]].append({'dosya_adi': dosya_adi, 'dosya_yolu': os.path.join(imageFolder, png_dosyasi)})

In [3]:
AgricultralMap = []
GreenAreaMap = []
IndustryMap = []
LivestockMap = []
MinesMap = []
RegionsMap = []
Cities = []

for x in konumlar.keys():
    Cities.append(x)
    y = konumlar[x]
    for d in y:
        dosya_adi = d.get("dosya_adi", "")

        if isinstance(dosya_adi, list):
            dosya_adi = dosya_adi[0] if dosya_adi else ""

        if "AgricultralMap" in dosya_adi:
            AgricultralMap.append(d["dosya_yolu"])
        elif "GreenAreaMap" in dosya_adi:
            GreenAreaMap.append(d["dosya_yolu"])
        elif "LivestockMap" in dosya_adi:
            LivestockMap.append(d["dosya_yolu"])
        elif "MinesMap" in dosya_adi:
            MinesMap.append(d["dosya_yolu"])
        elif "RegionsMap" in dosya_adi:
            RegionsMap.append(d["dosya_yolu"])

In [4]:
del konumlar

In [5]:
def GetImage(str_img):
    if(type(str_img) != type("")):
        return str_img
    img = cv2.imread(str_img, cv2.IMREAD_GRAYSCALE)
    return cv2.resize(img, (256, 256))

def ImgTiler(name):
    file_name, file_extension = os.path.splitext(os.path.basename(name))

    img = cv2.imread(name) 
    img_shape = img.shape
    tile_size = (384, 384)
    offset = (384, 384)
    
    dizin = debugImagesFolder + "/"+file_name+"/"
    
    folder_path = os.path.join(dizin, "")
    os.makedirs(folder_path, exist_ok=True)

    for i in range(int(math.ceil(img_shape[0]/(offset[1] * 1.0)))):
        for j in range(int(math.ceil(img_shape[1]/(offset[0] * 1.0)))):
            cropped_img = img[offset[1]*i:min(offset[1]*i+tile_size[1], img_shape[0]), offset[0]*j:min(offset[0]*j+tile_size[0], img_shape[1])]
            cv2.imwrite(dizin + file_name + "_" + str(i) + "_" + str(j) + ".png", cropped_img)

In [6]:
for i in range(len(AgricultralMap)):
    ImgTiler(AgricultralMap[i])
for i in range(len(GreenAreaMap)):
    ImgTiler(GreenAreaMap[i])
for i in range(len(LivestockMap)):
    ImgTiler(LivestockMap[i])
for i in range(len(MinesMap)):
    ImgTiler(MinesMap[i])
for i in range(len(RegionsMap)):
    ImgTiler(RegionsMap[i])

In [7]:
AgricultralMap.clear()
GreenAreaMap.clear()
IndustryMap.clear()
LivestockMap.clear()
MinesMap.clear()
RegionsMap.clear()

KlasorAdlari = ["AgricultralMap", "GreenAreaMap", "LivestockMap", "MinesMap", "RegionsMap"]

for ka in KlasorAdlari:
    klasorler = [klasor for klasor in os.listdir(debugImagesFolder) if klasor.startswith(ka)]
    for klasor in klasorler:
        png_dosyalari = [(debugImagesFolder + "/" + klasor + "/") + dosya for dosya in os.listdir(debugImagesFolder + "/" + klasor) if dosya.endswith('.png')]
        if ka == "AgricultralMap":
            AgricultralMap.extend(png_dosyalari)
        elif ka == "GreenAreaMap":
            GreenAreaMap.extend(png_dosyalari)
        elif ka == "LivestockMap":
            LivestockMap.extend(png_dosyalari)
        elif ka == "MinesMap":
            MinesMap.extend(png_dosyalari)
        elif ka == "RegionsMap":
            RegionsMap.extend(png_dosyalari)
            
del KlasorAdlari

In [8]:
for i in range(len(AgricultralMap)):
    AgricultralMap[i] = GetImage(AgricultralMap[i]) / 255
for i in range(len(GreenAreaMap)):
    GreenAreaMap[i] = GetImage(GreenAreaMap[i]) / 255
for i in range(len(LivestockMap)):
    LivestockMap[i] = GetImage(LivestockMap[i]) / 255
for i in range(len(MinesMap)):
    MinesMap[i] = GetImage(MinesMap[i]) / 255
for i in range(len(RegionsMap)):
    RegionsMap[i] = GetImage(RegionsMap[i]) / 255

In [9]:
data = {
    'AgricultralMap': AgricultralMap,
    'GreenAreaMap': GreenAreaMap,
    'LivestockMap': LivestockMap,
    'MinesMap': MinesMap
}

del AgricultralMap
del GreenAreaMap
del LivestockMap
del MinesMap

dataDF = pd.DataFrame(data)

In [10]:
target = {
    'RegionsMap': RegionsMap
}
targetDF = pd.DataFrame(target)
targetValDF = pd.DataFrame(data={'target': range(len(RegionsMap))})
targetMatrix = pd.DataFrame(data={"matrix": []})

del target
# targetValDF de yer alan değerler targetMatrix teki indexleri ifade eder.

In [11]:
np.unique(targetDF.iloc[0]["RegionsMap"])

zeroArray = np.zeros((256, 256))
oneArray = np.ones((256, 256))

targetMatrix.loc[0] = { "matrix" : zeroArray }
targetMatrix.loc[1] = { "matrix" : oneArray }

In [12]:
for index, row in targetDF.iterrows():
    unique_values = np.unique(row["RegionsMap"])
    
    if (0 in unique_values) and (len(unique_values) == 1):
        targetValDF.loc[index] = 0
    elif (1 in unique_values) and (len(unique_values) == 1):
        targetValDF.loc[index] = 1
    elif (len(unique_values) > 1):
        ones_count = np.count_nonzero(row["RegionsMap"] == 1)
        zeros_count = np.count_nonzero(row["RegionsMap"] == 0)
        
        sum_count = ones_count + zeros_count
        x = sum_count * 8 / 10
        
        if(ones_count > x):
            targetValDF.loc[index] = 1
        elif(zeros_count > x):
            targetValDF.loc[index] = 0
        else:
            targetMatrix.loc[len(targetMatrix)] = {"matrix": row["RegionsMap"]}
            targetValDF.loc[index] = (len(targetMatrix) - 1)

In [13]:
indexesToBeDeleted = []

# Veri olmayan boş bölgelerin silinmesi işlemi

for i in range(len(data["AgricultralMap"])):
    if(
        (len(np.unique(data["AgricultralMap"][i])) == 1 and (np.unique(data["AgricultralMap"][i]))[0] == 0) and
        (len(np.unique(data["GreenAreaMap"][i])) == 1 and (np.unique(data["GreenAreaMap"][i]))[0] == 0) and
        (len(np.unique(data["LivestockMap"][i])) == 1 and (np.unique(data["LivestockMap"][i]))[0] == 0) and
        (len(np.unique(data["MinesMap"][i])) == 1 and (np.unique(data["MinesMap"][i]))[0] == 0) and
        (len(np.unique(targetDF["RegionsMap"][i])) == 1 and (np.unique(targetDF["RegionsMap"][i]))[0] == 0)
    ):
        indexesToBeDeleted.append(i)

print("Toplam veri: " + str(len(data["AgricultralMap"])) + "\nSilinecek veri: " + str(len(indexesToBeDeleted)))

Toplam veri: 3330
Silinecek veri: 1857


In [14]:
numberOfDataToRemain = len(data["AgricultralMap"]) - len(indexesToBeDeleted)
#Veri bulunmayan kısımların çoğunu çıkartacağız ama burada veri bulunanların toplamının 10 da 1 i kadar veri bulunmayan bırakacağız.
#deleteNum = (numberOfDataToRemain * 1) / 50
#deleteNum = indexesToBeDeleted
import random
random.shuffle(indexesToBeDeleted)

#deleteNum = int(min(deleteNum, len(indexesToBeDeleted)))

#indexesToBeDeleted = indexesToBeDeleted[deleteNum:]

dataDF.drop(indexesToBeDeleted, inplace=True)
targetDF.drop(indexesToBeDeleted, inplace=True)
targetValDF.drop(indexesToBeDeleted, inplace=True)

dataDF.reset_index(drop=True, inplace=True)
targetDF.reset_index(drop=True, inplace=True)
targetValDF.reset_index(drop=True, inplace=True)

In [15]:
targetValDF.iloc[0]
targetMatrix.iloc[0]
print("Toplam veri: " + str(len(dataDF)) + "\nSilinecek veri: " + str(len(indexesToBeDeleted)))
dataDF.iloc[0]

different = []
for i in range(len(targetValDF)):
    different.append(targetValDF.iloc[i]["target"])

print(different)

Toplam veri: 1473
Silinecek veri: 1857
[2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 9, 10, 11, 12, 1, 13, 14, 0, 0, 0, 0, 0, 15, 16, 17, 0, 0, 0, 0, 0, 18, 0, 0, 0, 1, 0, 19, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 1, 21, 22, 23, 24, 1, 0, 0, 25, 26, 0, 0, 0, 0, 0, 27, 1, 1, 0, 28, 29, 0, 0, 0, 0, 30, 31, 1, 32, 33, 0, 0, 0, 0, 0, 0, 0, 0, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 36, 0, 0, 0, 0, 0, 1, 0, 37, 38, 39, 40, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 42, 43, 1, 0, 0, 0, 0, 0, 44, 45, 46, 47, 48, 49, 0, 0, 0, 0, 0, 1, 50, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 51, 1, 0, 0, 0, 0, 0, 0, 0, 52, 53, 0, 0, 54, 0, 0, 0, 55, 56, 0, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 58, 1, 59, 0, 0, 60, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 62, 63, 0, 0, 0, 0, 0, 64, 0, 0, 0, 0, 65, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We make a array like this:
```
[AgricultralMap, GreenAreaMap, LivestockMap, MinesMap] 
```

In [16]:
UnifiedMap = []
for i in range(len(dataDF["AgricultralMap"])):
    Arr = []
    Arr.append(dataDF["AgricultralMap"][i].tolist())
    Arr.append(dataDF["GreenAreaMap"][i].tolist())
    Arr.append(dataDF["LivestockMap"][i].tolist())
    Arr.append(dataDF["MinesMap"][i].tolist())
    UnifiedMap.append(Arr)

len(UnifiedMap)
del dataDF

In [17]:
np.shape(UnifiedMap[0])

(4, 256, 256)

In [18]:
for i in range(len(UnifiedMap)):
    Arr = []
    for y in range(len(UnifiedMap[i][0])):
        ArrY = []
        for x in range(len(UnifiedMap[i][0][y])):
            ArrY.append([UnifiedMap[i][0][y][x], UnifiedMap[i][1][y][x], UnifiedMap[i][2][y][x], UnifiedMap[i][3][y][x]])
        Arr.append(ArrY)
    UnifiedMap[i] = Arr

In [19]:
np.shape(UnifiedMap[0])

(256, 256, 4)

In [20]:
np.shape(UnifiedMap)

(1473, 256, 256, 4)

# AI

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
X = UnifiedMap
y = np.array(targetValDF["target"].tolist())

nX = np.array(X)
ny = np.array(y)

In [23]:
# Ram
del UnifiedMap
del RegionsMap
del data
del X

In [24]:
X_train, X_test, y_train, y_test = train_test_split(nX, ny, test_size=0.2, random_state=42)

In [25]:
X_train.shape

(1178, 256, 256, 4)

In [32]:
vallen = len(targetValDF)

model = models.Sequential()
model.add(layers.Conv2D(32, (4, 4), activation='relu', input_shape=(256, 256, 4)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(vallen, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 253, 253, 32)      2080      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 30, 30, 64)       

In [33]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 253, 253, 32)      2080      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 126, 126, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 30, 30, 64)       

In [34]:
history = model.fit(nX, ny, epochs=10, batch_size=32, validation_split=0.2, shuffle=True)

Epoch 1/10
37/37 [==============================] - 13s 318ms/step - loss: 3.4944 - accuracy: 0.6732 - val_loss: 3.9055 - val_accuracy: 0.5085
Epoch 2/10
37/37 [==============================] - 12s 322ms/step - loss: 2.2833 - accuracy: 0.6927 - val_loss: 4.5014 - val_accuracy: 0.5085
Epoch 3/10
37/37 [==============================] - 11s 304ms/step - loss: 2.1132 - accuracy: 0.7029 - val_loss: 4.6302 - val_accuracy: 0.5593
Epoch 4/10
37/37 [==============================] - 11s 308ms/step - loss: 2.0752 - accuracy: 0.7063 - val_loss: 5.0076 - val_accuracy: 0.5085
Epoch 5/10
37/37 [==============================] - 11s 305ms/step - loss: 1.9836 - accuracy: 0.7012 - val_loss: 5.3847 - val_accuracy: 0.5119
Epoch 6/10
37/37 [==============================] - 13s 346ms/step - loss: 1.7199 - accuracy: 0.7097 - val_loss: 6.8268 - val_accuracy: 0.5525
Epoch 7/10
37/37 [==============================] - 11s 306ms/step - loss: 1.2258 - accuracy: 0.7623 - val_loss: 9.3279 - val_accuracy: 0.5254

In [35]:
from keras.models import save_model
save_model(model, 'Model/4ArrCNNModel.keras')

In [36]:
#X_test = X_test.reshape(len(X_test), 256, 256, 4)

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print(test_acc)

10/10 - 1s - loss: 3.3528 - accuracy: 0.8339 - 560ms/epoch - 56ms/step
0.8338983058929443


In [37]:
for i in range(len(targetMatrix["matrix"])):
    targetMatrix["matrix"][i] =  [[1 if val != 0 else 0 for val in row] for row in targetMatrix["matrix"][i]]

np.save("Model/targetMatrix.npy", targetMatrix)
targetMatrix.to_json("Model/targetMatrix.json", orient='records')

targetMatrix.head(15)

,matrix
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
